In [1]:
import sys
 
sys.path.append('..')
from piece import piece

piece.start(should_send_to_score=False)

In [108]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools import note, spectrum
from soundmining_tools.generative import *
from soundmining_tools.sieve import *
from soundmining_tools.modular.control_instruments import ControlInstrument
from soundmining_tools.modular.instrument import NodeId
import math
from typing import Callable

from soundmining_tools.sequencer import Sequencer, SequenceNote
from enum import Enum
from soundmining_tools import supercollider_client
from soundmining_tools.modular import instrument
import random

piece.reset()
piece.synth_player \
    .start()

class NoiseNoteHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)        

    
    def handle_note(self, patch_arguments: PatchArguments) -> None:
    
                            
class MyHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)        

    def handle_note(self, patch_arguments: PatchArguments) -> None:            
        pass                
            
        

my_handler = MyHandler(piece.supercollider_client)
piece.receiver.set_note_handler(my_handler)                    


In [1]:
piece.stop()


NameError: name 'piece' is not defined